In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors 
from shapely.geometry import Point, shape, Polygon, MultiPolygon
from shapely import geometry
from shapely.ops import unary_union
from srai.loaders.osm_loaders import OSMOnlineLoader
from srai.loaders.osm_loaders.filters.popular import get_popular_tags
from functional import seq
from matplotlib import colormaps
from srai.constants import WGS84_CRS
from srai.plotting.folium_wrapper import plot_regions
from srai.regionalizers import VoronoiRegionalizer, geocode_to_region_gdf, AdministrativeBoundaryRegionalizer
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors 

import cartopy.crs as ccrs
import cartopy.feature as cfeature


import branca.colormap as cm_branca
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions import  make_PDF, make_lognormal_PDF
import voronoi_functions as vf 

from datetime import datetime, timedelta
import xarray as xr

# set plotscale
plt.style.use('../python_style_Meike.mplstyle')

In [ ]:
# settings
particle_types = ['tracer','tracer_random','inertial_SM_drag_Rep','inertial_SM_Rep_constant']
coriolislist = [True,False]
B =B = 0.68
tau = 2994.76
T = 719 
gradient = True
runtime = timedelta(days=30)
starttimes = [datetime(2023, 9, 1),
              datetime(2023, 10, 1),
              datetime(2023, 11, 1),
              datetime(2023, 12, 1),
              datetime(2024, 1, 1),
              datetime(2024, 2, 1)]
starttime = datetime(2023, 9, 1)
endtime = starttime + runtime
displacement = 300
loc = "NWES"
land_handling = "anti_beaching"
Rep = 0
Replist = [0,10,100,1000]
time_resolution_list = ['hourly','daily']




base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'

basefile_tracer =outputfile_tracer = (
      base_directory + "{particle_type}/voronoi_data/{loc}_{time_resolution}_"
      "start{y_s:04d}_{m_s:02d}_{d_s:02d}_T{T:04d}h.geojson"
)

basefile_tracer_random = (
      base_directory + "{particle_type}/voronoi_data/{loc}_{time_resolution}_"
      "start{y_s:04d}_{m_s:02d}_{d_s:02d}_T{T:04d}h_{d_s:02d}.parquet"
)

basefile_MR = (
      base_directory + "{particle_type}/voronoi_data/{loc}_{time_resolution}_"
      "start{y_s:04d}_{m_s:02d}_{d_s:02d}_T{T:04d}h_"
      "B{B:04d}_tau{tau:04d}_cor_{coriolis}_gradient_{gradient}.geojson"
)

basefile_MR_Rep_constant = (
      base_directory + "{particle_type}/voronoi_data/{loc}_{time_resolution}_"
      "start{y_s:04d}_{m_s:02d}_{d_s:02d}_T{T:04d}h_Rep_{Rep:04d}"
      "B{B:04d}_tau{tau:04d}_cor_{coriolis}_gradient_{gradient}.geojson"
)

inputfiles = {'tracer':basefile_tracer,
              'tracer_random':basefile_tracer,
              'inertial_SM_drag_Rep':basefile_MR,
              'inertial_SM_Rep_constant':basefile_MR_Rep_constant,
              }

In [ ]:
# settings 
#temporary
# particle_types = ['tracer','tracer_random','inertial_SM_Rep_constant']
# Tres = 'hourly'
data = {}
for pt in particle_types:
    data[pt]={}
    for Tres in time_resolution_list:
        if(pt in ('tracer', 'tracer_random')):
            data[pt][Tres]={}
            data[pt][Tres][True]={}
            data[pt][Tres][True][None]={}
        elif(pt == 'inertial_SM_drag_Rep'):
            data[pt][Tres]={}
            data[pt][Tres][True]={}
            data[pt][Tres][True][None]={}
            # for coriolis in coriolislist:
            #     data[pt][Tres]={}
            #     data[pt][Tres][coriolis]={}
            #     data[pt][Tres][coriolis][None]={}
        elif(pt == 'inertial_SM_Rep_constant'):
            data[pt][Tres]={}
            data[pt][Tres][True]={}
            data[pt][Tres][True][None]={}
            for Rep in Replist:
                
                data[pt][Tres][True][Rep]={}
        else:
            for coriolis in coriolislist:
                data[pt][Tres]={}
                data[pt][Tres][coriolis]={}
                data[pt][Tres][coriolis][None]={}
print(data)

for pt in particle_types:
    print(pt)
    for Tres in time_resolution_list:
        for starttime in starttimes:
            # print(starttime)
            endtime = starttime + runtime 
            date = f'{starttime.year:04d}/{starttime.month:02d}'
            if(pt == 'tracer'):
                inputfile = inputfiles[pt].format(
                    loc=loc,
                    y_s=starttime.year,
                    m_s=starttime.month,
                    d_s=starttime.day,
                    y_e=endtime.year,
                    m_e=endtime.month,
                    d_e=endtime.day,
                    land_handling=land_handling,
                    particle_type=pt,
                    time_resolution=Tres,
                    T=T
                )
                data[pt][Tres][True][None][date] = gpd.GeoDataFrame.from_file(inputfile)
            elif(pt == 'tracer_random'):
                inputfile = inputfiles[pt].format(
                    loc=loc,
                    y_s=starttime.year,
                    m_s=starttime.month,
                    d_s=starttime.day,
                    y_e=endtime.year,
                    m_e=endtime.month,
                    d_e=endtime.day,
                    land_handling=land_handling,
                    particle_type=pt,
                    time_resolution=Tres,
                    T=T
                )
                test = gpd.GeoDataFrame.from_file(inputfile)
                data[pt][Tres][True][None][date] = test
            elif(pt == 'inertial_SM_drag_Rep'):
                for coriolis in [True]:
                    inputfile = inputfiles[pt].format(
                        loc=loc,
                        y_s=starttime.year,
                        m_s=starttime.month,
                        d_s=starttime.day,
                        y_e=endtime.year,
                        m_e=endtime.month,
                        d_e=endtime.day,
                        land_handling=land_handling,
                        coriolis=coriolis,
                        particle_type=pt,
                        d = displacement,
                        Rep = Rep,
                        gradient = gradient,
                        B = int(1000*B),
                        tau = int(tau),
                        time_resolution=Tres,
                        T=T
                    )
                    data[pt][Tres][coriolis][None][date] = gpd.GeoDataFrame.from_file(inputfile)
            else:
                for Rep in Replist:
                    inputfile = inputfiles[pt].format(
                        loc=loc,
                        y_s=starttime.year,
                        m_s=starttime.month,
                        d_s=starttime.day,
                        y_e=endtime.year,
                        m_e=endtime.month,
                        d_e=endtime.day,
                        land_handling=land_handling,
                        coriolis=True,
                        particle_type=pt,
                        d = displacement,
                        Rep = Rep,
                        gradient = gradient,
                        time_resolution=Tres,
                        B = int(1000*B),
                        tau = int(tau),
                        T=T
                    )
                    # print(inputfile)
                    data[pt][Tres][True][Rep][date] = gpd.GeoDataFrame.from_file(inputfile)

In [ ]:
starttime=starttimes[0]
date = f'{starttime.year:04d}/{starttime.month:02d}'
Tres='hourly'
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()},figsize=(12,8))
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['tracer'][Tres][True][None][date],colormap = cm.magma,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1, 
             colorbar_on =True)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0., linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 15}
gl.ylabel_style =  {'size': 15}
# ax.set_frame_on(False)
fig.tight_layout()
# fig.savefig('../figures/voronoi_analysis/voronoi_tesselation_NWES_tracer_hourly_field_september.pdf')


In [ ]:
starttime=starttimes[0]
date = f'{starttime.year:04d}/{starttime.month:02d}'
Tres='hourly'
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()},figsize=(12,8))
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['inertial_SM_Rep_constant'][Tres][True][0][date],colormap = cm.magma,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1, 
             colorbar_on =True)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0., linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 15}
gl.ylabel_style =  {'size': 15}
ax.set_title('Re$_p$ = 0, 2023-09-30 23:00:00')
# ax.set_frame_on(False)
fig.tight_layout()
# fig.savefig('../figures/voronoi_analysis/voronoi_tesselation_NWES_tracer_hourly_field_september.pdf')

In [ ]:
starttime=starttimes[0]
date = f'{starttime.year:04d}/{starttime.month:02d}'
Tres='hourly'
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()},figsize=(12,8))
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['inertial_SM_Rep_constant'][Tres][True][1000][date],colormap = cm.magma,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1, 
             colorbar_on =True)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0., linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 15}
gl.ylabel_style =  {'size': 15}
ax.set_title('Re$_p$ = 1000, 2023-09-30 23:00:00')
# ax.set_frame_on(False)
fig.tight_layout()
# fig.savefig('../figures/voronoi_analysis/voronoi_tesselation_NWES_tracer_hourly_field_september.pdf')

In [ ]:
# averaged over all months 
Blues = colormaps['magma']
starttime=starttimes[0]

bluelist = Blues(np.linspace(0.2, 0.9, 4))

fig, ax = plt.subplots()
Tres='hourly'


data_arrays= list(data['tracer'][Tres][True][None].values())
density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
combined_list = np.array([item for arr in density_per_particle for item in arr])
bins_tracer, pdf_tracer = make_lognormal_PDF(combined_list,norm=True, nbins = 100,vmin=0.5*1E-3,vmax=1E0)
ax.plot(bins_tracer,pdf_tracer,'-',color='black')

data_arrays= list(data['tracer_random'][Tres][True][None].values())
density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
combined_list = np.array([item for arr in density_per_particle for item in arr])
bins_random, pdf_random = make_lognormal_PDF(combined_list,norm=True, nbins = 100,vmin=0.5*1E-3,vmax=1E0)
ax.plot(bins_random,pdf_random,'-',color='grey')

for Rep, color in zip(Replist, bluelist):
    data_arrays= list(data['inertial_SM_Rep_constant'][Tres][True][Rep].values())
    density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
    combined_list = np.array([item for arr in density_per_particle for item in arr])
    bins, pdf = make_lognormal_PDF(combined_list,norm=True,nbins=100,vmin=0.5*1E-3,vmax=1E0)
    ax.plot(bins,pdf,'--',color=color)

data_arrays= list(data['inertial_SM_drag_Rep'][Tres][True][None].values())
density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
combined_list = np.array([item for arr in density_per_particle for item in arr])
bins_random, pdf_random = make_lognormal_PDF(combined_list,norm=True, nbins = 100,vmin=0.5*1E-3,vmax=1E0)
# ax.plot(bins_random,pdf_random,'-.',color='green')
    
ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('PDF')
# ax.set_title(Tres + ' field')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['tracer','tracer random','Re$_p$ = 0','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000','Re$_p$ flexible'])

fig.tight_layout()
fig.savefig('../figures/voronoi_analysis/av_hourly_field_PDF.pdf')

In [ ]:
# averaged over all months 
Blues = colormaps['Blues']
starttime=starttimes[0]

bluelist = Blues(np.linspace(0.4, 0.9, 4))

fig, ax = plt.subplots()
Tres='daily'


data_arrays= list(data['tracer'][Tres][True][None].values())
density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
combined_list = np.array([item for arr in density_per_particle for item in arr])
bins_tracer, pdf_tracer = make_lognormal_PDF(combined_list,norm=True, nbins = 100,vmin=0.5*1E-3,vmax=1E0)
ax.plot(bins_tracer,pdf_tracer,'-',color='black')

data_arrays= list(data['tracer_random'][Tres][True][None].values())
density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
combined_list = np.array([item for arr in density_per_particle for item in arr])
bins_random, pdf_random = make_lognormal_PDF(combined_list,norm=True, nbins = 100,vmin=0.5*1E-3,vmax=1E0)
ax.plot(bins_random,pdf_random,'-',color='grey')

for Rep, color in zip(Replist, bluelist):
    data_arrays= list(data['inertial_SM_Rep_constant'][Tres][True][Rep].values())
    density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
    combined_list = np.array([item for arr in density_per_particle for item in arr])
    bins, pdf = make_lognormal_PDF(combined_list,norm=True,nbins=100,vmin=0.5*1E-3,vmax=1E0)
    ax.plot(bins,pdf,'--',color=color)

data_arrays= list(data['inertial_SM_drag_Rep'][Tres][True][None].values())
density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
combined_list = np.array([item for arr in density_per_particle for item in arr])
bins_random, pdf_random = make_lognormal_PDF(combined_list,norm=True, nbins = 100,vmin=0.5*1E-3,vmax=1E0)
ax.plot(bins_random,pdf_random,'-.',color='firebrick')
    
ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('PDF')
ax.set_title(Tres + ' field')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['tracer','tracer random','Re$_p$ = 0','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000','Re$_p$ flexible'])

fig.tight_layout()
fig.savefig('../figures/voronoi_analysis/av_daily_field_PDF.pdf')

In [ ]:


fig, ax = plt.subplots()
Tres= 'daily'

data_arrays= list(data['tracer'][Tres][True][None].values())
density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
combined_list = np.array([item for arr in density_per_particle for item in arr])
bins_tracer, pdf_tracer = make_lognormal_PDF(combined_list,norm=True, nbins = 100,vmin=1E-3,vmax=1E0)
ax.plot(bins_tracer,pdf_tracer,'-',color='black',zorder=-20)

data_arrays= list(data['tracer_random'][Tres][True][None].values())
density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
combined_list = np.array([item for arr in density_per_particle for item in arr])
bins_random, pdf_random = make_lognormal_PDF(combined_list,norm=True, nbins = 100,vmin=1E-3,vmax=1E0)
ax.plot(bins_random,pdf_random,'-',color='grey',zorder=-20)

data_arrays= list(data['inertial_SM_drag_Rep'][Tres][True][None].values())
density_per_particle = [vf.make_value_list_per_particle(element,'density') for element in data_arrays]
combined_list = np.array([item for arr in density_per_particle for item in arr])
bins_random, pdf_random = make_lognormal_PDF(combined_list,norm=True, nbins = 100,vmin=1E-3,vmax=1E0)
ax.plot(bins_random,pdf_random,'-.',color='firebrick',zorder=-20)

ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('PDF')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['tracer','displaced tracer','flexible Re$_p$'])


In [ ]:
np.array(combined_list).shape

In [ ]:
Blues = colormaps['magma']
starttime=starttimes[2]
date = f'{starttime.year:04d}/{starttime.month:02d}'


# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.2, 0.9, 4))

fig, ax = plt.subplots()

Tres ='hourly'
# density = data['tracer'][True][None]['density'].values
density_per_particle = vf.make_value_list_per_particle(data['tracer'][Tres][True][None][date],'density')
bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 50,vmin=1E-3,vmax=1E0)
ax.plot(bins_tracer,pdf_tracer,'-',color='black',zorder=-20)

density_per_particle = vf.make_value_list_per_particle(data['tracer_random'][Tres][True][None][date],'density')
bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 50,vmin=1E-3,vmax=1E0)
ax.plot(bins_tracer,pdf_tracer,'-',color='grey',zorder=-20)


density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][0][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins,pdf,'--',color=bluelist[0])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][10][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins,pdf,'--',color=bluelist[1])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][100][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins,pdf,'--',color=bluelist[2])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][1000][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins,pdf,'--',color=bluelist[3])


density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_drag_Rep'][Tres][True][None][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
# ax.plot(bins,pdf,'--',color='firebrick')
print(np.sum(pdf))
ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('PDF')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['tracer','tracer random','Re$_p$ = 0','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000','Re$_p$ flexible'])




In [ ]:
#cdf 

Blues = colormaps['Blues']
starttime=starttimes[2]
date = f'{starttime.year:04d}/{starttime.month:02d}'


# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9, 4))

fig, ax = plt.subplots()

Tres ='daily'
# density = data['tracer'][True][None]['density'].values
density_per_particle = vf.make_value_list_per_particle(data['tracer'][Tres][True][None][date],'density')
bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 50,vmin=1E-3,vmax=1E0)
cdf_tracer = np.cumsum(pdf_tracer)
ax.plot(bins_tracer,cdf_tracer,'-',color='black',zorder=-20)

density_per_particle = vf.make_value_list_per_particle(data['tracer_random'][Tres][True][None][date],'density')
bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 50,vmin=1E-3,vmax=1E0)
cdf_tracer = np.cumsum(pdf_tracer)
ax.plot(bins_tracer,cdf_tracer,'-',color='grey',zorder=-20)


density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][0][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf,'--',color=bluelist[0])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][10][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf,'--',color=bluelist[1])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][100][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf,'--',color=bluelist[2])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][1000][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf,'--',color=bluelist[3])


density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_drag_Rep'][Tres][True][None][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf,'--',color='firebrick')
ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('CDF')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['tracer','tracer random','Re$_p$ = 0','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000','Re$_p$ flexible'])




In [ ]:
#cdf 

Blues = colormaps['Blues']
starttime=starttimes[2]
date = f'{starttime.year:04d}/{starttime.month:02d}'


# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9, 4))

fig, ax = plt.subplots()

Tres ='daily'
# density = data['tracer'][True][None]['density'].values
density_per_particle = vf.make_value_list_per_particle(data['tracer'][Tres][True][None][date],'density')
bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 50,vmin=1E-3,vmax=1E0)
cdf_tracer = np.cumsum(pdf_tracer)
# ax.plot(bins_tracer,cdf_tracer,'-',color='black',zorder=-20)

density_per_particle = vf.make_value_list_per_particle(data['tracer_random'][Tres][True][None][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True, nbins = 50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins, cdf-cdf_tracer,'-',color='grey',zorder=-20)


density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][0][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins, cdf-cdf_tracer,'--',color=bluelist[0])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][10][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins, cdf-cdf_tracer,'--',color=bluelist[1])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][100][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins, cdf-cdf_tracer,'--',color=bluelist[2])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][1000][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins, cdf-cdf_tracer,'--',color=bluelist[3])


density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_drag_Rep'][Tres][True][None][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
cdf = np.cumsum(pdf)
ax.plot(bins, cdf-cdf_tracer,'--',color='firebrick')
ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('CDF - CDF tracer')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['tracer random','Re$_p$ = 0','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000','Re$_p$ flexible'])




In [ ]:

Blues = colormaps['Blues']
starttime=starttimes[0]
date = f'{starttime.year:04d}/{starttime.month:02d}'


# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9, 4))

fig, ax = plt.subplots()

Tres ='hourly'
# density = data['tracer'][True][None]['density'].values
density_per_particle = vf.make_value_list_per_particle(data['tracer'][Tres][True][None][date],'area')
bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 500,vmin=1E-5,vmax=1E0)
# ax.plot(bins_tracer,pdf_tracer,'-',color='black',zorder=-20)

density_per_particle = vf.make_value_list_per_particle(data['tracer_random'][Tres][True][None][date],'area')
bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 100,vmin=1E-5,vmax=1E0)
ax.plot(bins_tracer,pdf_tracer,'-',color='grey',zorder=-20)


density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][0][date],'area')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=100,vmin=1E-5,vmax=1E0)
ax.plot(bins,pdf,'--',color=bluelist[0])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][10][date],'area')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=100,vmin=1E-5,vmax=1E0)
ax.plot(bins,pdf,'--',color=bluelist[1])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][100][date],'area')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=100,vmin=1E-5,vmax=1E0)
ax.plot(bins,pdf,'--',color=bluelist[2])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][1000][date],'area')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=100,vmin=1E-5,vmax=1E0)
ax.plot(bins,pdf,'--',color=bluelist[3])


density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_drag_Rep'][Tres][True][None][date],'area')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=100,vmin=1E-5,vmax=1E0)
ax.plot(bins,pdf,'--',color='firebrick')
print(np.sum(pdf))
ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('PDF')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['tracer','tracer random','Re$_p$ = 0','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000','Re$_p$ flexible'])




In [ ]:
Blues = colormaps['Blues']

bluelist = Blues(np.linspace(0.4, 0.9, 4))
fig, ax = plt.subplots()
i=0
legend = []
for starttime in starttimes:
    date = f'{starttime.year:04d}/{starttime.month:02d}'
    density_per_particle = vf.make_value_list_per_particle(data['tracer'][Tres][True][None][date],'density')
    bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 100,vmin=1E-3,vmax=1E0)
    ax.plot(bins_tracer,pdf_tracer,'-',zorder=-20,color=bluelist[i])
    density_per_particle = vf.make_value_list_per_particle(data['tracer_random'][Tres][True][None][date],'density')
    bins, pdf = make_lognormal_PDF(density_per_particle,norm=True, nbins = 100,vmin=1E-3,vmax=1E0)
    ax.plot(bins,pdf,'--',zorder=-20,color=bluelist[i])
    legend.append(date)
    i+=1
ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('PDF')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
# ax.legend(legend)

In [ ]:
Blues = colormaps['Blues']
starttime=starttimes[1]
date = f'{starttime.year:04d}/{starttime.month:02d}'
density_per_hexagon = 1/36.129062164	
norm = density_per_hexagon 

# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9, 4))

fig, ax = plt.subplots()

Tres ='hourly'
# density = data['tracer'][True][None]['density'].values
density_per_particle = vf.make_value_list_per_particle(data['tracer'][Tres][True][None][date],'density')
bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 50,vmin=1E-3,vmax=1E0)
# ax.plot(bins_tracer,pdf_tracer,'-',color='black',zorder=-20)

density_per_particle = vf.make_value_list_per_particle(data['tracer_random'][Tres][True][None][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True, nbins = 50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'-',color='grey')

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][0][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'--',color=bluelist[0])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][10][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'--',color=bluelist[1])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][100][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'--',color=bluelist[2])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][1000][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'--',color=bluelist[3])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_drag_Rep'][Tres][True][None][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'-.',color='firebrick')
print(np.sum(pdf))
ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('PDF - PDF tracer')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['tracer random','Re$_p$ = 0','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000','Re$_p$ flexible'])
ax.set_title(Tres + ' field')



In [ ]:
Blues = colormaps['Blues']
starttime=starttimes[1]
date = f'{starttime.year:04d}/{starttime.month:02d}'
density_per_hexagon = 1/36.129062164	
norm = density_per_hexagon 

# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9, 4))

fig, ax = plt.subplots()

Tres ='daily'
# density = data['tracer'][True][None]['density'].values
density_per_particle = vf.make_value_list_per_particle(data['tracer'][Tres][True][None][date],'density')
bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 50,vmin=1E-3,vmax=1E0)
# ax.plot(bins_tracer,pdf_tracer,'-',color='black',zorder=-20)

density_per_particle = vf.make_value_list_per_particle(data['tracer_random'][Tres][True][None][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True, nbins = 50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'-',color='grey')

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][0][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'--',color=bluelist[0])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][10][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'--',color=bluelist[1])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][100][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'--',color=bluelist[2])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][1000][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'--',color=bluelist[3])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_drag_Rep'][Tres][True][None][date],'density')
bins, pdf = make_lognormal_PDF(density_per_particle,norm=True,nbins=50,vmin=1E-3,vmax=1E0)
ax.plot(bins/norm,pdf-pdf_tracer,'-.',color='firebrick')
print(np.sum(pdf))
ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('PDF - PDF tracer')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['tracer random','Re$_p$ = 0','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000','Re$_p$ flexible'])
ax.set_title(Tres + ' field')


In [ ]:
Blues = colormaps['Blues']
starttime=starttimes[1]
date = f'{starttime.year:04d}/{starttime.month:02d}'
density_per_hexagon = 1# 36.129062164	

# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9, 4))

fig, ax = plt.subplots()
Tres = 'hourly'

# density = data['tracer'][True][None]['density'].values
density_per_particle = vf.make_value_list_per_particle(data['tracer'][Tres][True][None][date],'area')
bins_tracer, pdf_tracer = make_PDF(density_per_particle,norm=True, nbins = 2000,vmin=0,vmax=1)
ax.plot(bins_tracer/density_per_hexagon,pdf_tracer,'-',color='black')

density_per_particle = vf.make_value_list_per_particle(data['tracer_random'][Tres][True][None][date],'area')
bins_tracer, pdf_tracer = make_PDF(density_per_particle,norm=True, nbins = 2000,vmin=0,vmax=1)
ax.plot(bins_tracer/density_per_hexagon,pdf_tracer,'-',color='grey')

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][0][date],'area')
bins, pdf = make_PDF(density_per_particle,norm=True,nbins=2000,vmin=0,vmax=1)
ax.plot(bins/density_per_hexagon,pdf,'--',color=bluelist[0])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][10][date],'area')
bins, pdf = make_PDF(density_per_particle,norm=True, nbins = 2000,vmin=0,vmax=1)
ax.plot(bins/density_per_hexagon,pdf,'--',color=bluelist[1])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][100][date],'area')
bins, pdf = make_PDF(density_per_particle,norm=True, nbins = 2000,vmin=0,vmax=1)
ax.plot(bins/density_per_hexagon,pdf,'--',color=bluelist[2])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][Tres][True][1000][date],'area')
bins, pdf = make_PDF(density_per_particle,norm=True, nbins = 2000,vmin=0,vmax=1)
ax.plot(bins/density_per_hexagon,pdf,'--',color=bluelist[3])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_drag_Rep'][Tres][True][None][date],'area')
bins, pdf = make_PDF(density_per_particle,norm=True, nbins = 2000,vmin=0,vmax=1)
ax.plot(bins/density_per_hexagon,pdf,'--',color='firebrick')
print(np.sum(pdf))
ax.set_xscale('log')
# ax.set_xlabel('particle density [1/km$^2$]')
ax.set_xlabel('particle density / homogenous starting density')
ax.set_ylabel('PDF')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['tracer','Re$_p$ = 0','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000'])

In [ ]:
Blues = colormaps['Blues']

density_per_hexagon = 1/36.129062164	

# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9, 4))

fig, ax = plt.subplots()


# density = data['tracer'][True][None]['density'].values
density_per_particle = vf.make_value_list_per_particle(data['tracer'][True][None],'density')
bins_tracer, pdf_tracer = make_PDF(density_per_particle,norm=True, nbins = 1000,vmin=0,vmax=1)
# ax.plot(bins_tracer/density_per_hexagon,pdf_tracer,'-',color='black',zorder=-20)

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][True][0],'density')
bins, pdf = make_PDF(density_per_particle,norm=True,nbins=1000,vmin=0,vmax=1)
ax.plot(bins/density_per_hexagon,pdf-pdf_tracer,'--',color=bluelist[0])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][True][10],'density')
bins, pdf = make_PDF(density_per_particle,norm=True, nbins = 1000,vmin=0,vmax=1)
ax.plot(bins/density_per_hexagon,pdf-pdf_tracer,'--',color=bluelist[1])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][True][100],'density')
bins, pdf = make_PDF(density_per_particle,norm=True, nbins = 1000,vmin=0,vmax=1)
ax.plot(bins/density_per_hexagon,pdf-pdf_tracer,'--',color=bluelist[2])

density_per_particle = vf.make_value_list_per_particle(data['inertial_SM_Rep_constant'][True][1000],'density')
bins, pdf = make_PDF(density_per_particle,norm=True, nbins = 1000,vmin=0,vmax=1)
ax.plot(bins/density_per_hexagon,pdf-pdf_tracer,'--',color=bluelist[3])

ax.set_xscale('log')
# ax.set_xlabel('particle density [1/km$^2$]')
ax.set_xlabel('particle density / homogenous starting density')
ax.set_ylabel('PDF')
# ax.set_xlim(0,2)
# ax.set_yscale('log')
ax.legend(['Re$_p$ = 0','Re$_p$ = 10','Re$_p$ = 100','Re$_p$ = 1000'])

In [ ]:
fig, ax =plt.subplots()



duplicates=data['tracer'][True][None]['duplicates'].values
# plt.plot(duplicates)
max_number = int(np.max(duplicates))
density= data['tracer'][True][None]['density'].values
density_per_particle = density
for n in range(2,max_number+1):
    density_per_particle=np.concatenate((density_per_particle,density[duplicates==n]))
    for m in range(2,int(n)):
        density_per_particle=np.concatenate((density_per_particle,density[duplicates==n]))

bins_tracer, pdf_tracer = make_lognormal_PDF(density_per_particle,norm=True, nbins = 100,vmin=1E-3,vmax=1E4)
ax.plot(bins_tracer,pdf_tracer,'-',color='black',zorder=-20)




In [ ]:
Blues = colormaps['Blues']



# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9,2))

fig, ax = plt.subplots(figsize=(10,5))

density = data['tracer'][True][None]['density'].values
bins_tracer, pdf_tracer = make_lognormal_PDF(density,norm=True, nbins = 100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf_tracer = np.cumsum(pdf_tracer)
# ax.plot(bins,cdf_tracer,'-',color='black',zorder=-20)

density = data['inertial_SM_Rep_constant'][True][100]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[0])

# density = data['inertial_SM_Rep_constant'][True][100]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[1])

# density = data['inertial_SM_Rep_constant'][True][1000]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[2])

density = data['inertial_SM_drag_Rep'][True][None]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[1])

# density = data['inertial_SM_drag_Rep'][False][None]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[2])


ax.set_xscale('log')
ax.set_xlabel('Particle density [1/km$^2$]')
ax.set_ylabel('CDF - tracer CDF')
ax.legend(['Re$_p$ =100','Re$_p$ flexible'],handlelength=1.5)

fig.tight_layout()
fig.savefig('../figures/poster/CDF.pdf')

In [ ]:
Blues = colormaps['Blues']



# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.4, 0.9,2))

fig, ax = plt.subplots()

density = data['tracer'][True][None]['density'].values
bins_tracer, pdf_tracer = make_lognormal_PDF(density,norm=True, nbins = 100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf_tracer = np.cumsum(pdf_tracer)
ax.plot(bins_tracer,cdf_tracer,'-',color='black',zorder=-20)

density = data['inertial_SM_Rep_constant'][True][100]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf,'--',color=bluelist[0])

# density = data['inertial_SM_Rep_constant'][True][100]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[1])

# density = data['inertial_SM_Rep_constant'][True][1000]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[2])

density = data['inertial_SM_drag_Rep'][True][None]['density'].values
bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf = np.cumsum(pdf)
ax.plot(bins,cdf,'--',color=bluelist[1])

# density = data['inertial_SM_drag_Rep'][False][None]['density'].values
# bins, pdf = make_lognormal_PDF(density,norm=True,nbins=100,vmin=1E-3,vmax=1E6)
# bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
# bin_width = np.diff(bin_edges)
# cdf = np.cumsum(pdf)
# ax.plot(bins,cdf-cdf_tracer,'--',color=bluelist[2])


ax.set_xscale('log')
ax.set_xlabel('particle density [1/km$^2$]')
ax.set_ylabel('CDF')

In [ ]:

#1E-3,vmax=1E6
bin_edges = np.logspace(np.log10(1E-3), np.log10(1E6), 100 + 1)
bin_width = np.diff(bin_edges)
cdf = np.cumsum(pdf_tracer)
fig,ax=plt.subplots()
ax.plot(bins_tracer,cdf)
ax.set_xscale('log')

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['tracer'][True][None],colormap = cm.Blues,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1)

# gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#             linewidth=0, color='gray', alpha=0.5, linestyle='--')
# gl.top_labels = False
# gl.right_labels = False
# gl.xlabel_style = {'size': 18}
# gl.ylabel_style =  {'size': 18}
ax.set_xlim(-14.6,-10.5)
ax.set_ylim(54.4,56.1)
ax.set_frame_on(False)
fig.tight_layout()
fig.savefig('../figures/poster/voronoi_tesselation_zoomed.pdf')

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['inertial_SM_Rep_constant'][True][100],colormap = cm.Blues,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}
# ax.set_xlim(-14.6,-10.5)
# ax.set_ylim(54.4,56.1)
ax.set_xlim(-4,2)
ax.set_ylim(48.5,51.5)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
ax.add_feature(cfeature.LAND, edgecolor="black", color="lightgray")
vf.plot_voronoi(fig, ax , data['inertial_SM_drag_Rep'][True][None],colormap = cm.Blues,
             colormap_scale = 'log',
             color_scale_type = 'density',
             vmin = 1E-3,
             vmax = 1E1)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}
ax.set_xlim(-14.6,-10.5)
ax.set_ylim(54.4,56.1)